In [1]:
from diffeq import *

## symbols

In [2]:
from diffeq.utils.symbolic import *
x, y = variable('x'), variable('y')
expression = x**2 + y*x + cos(x) + 3

print(expression)

print('без оптимизаций:')
print(expression.diff('x'))
print(expression.diff('y'))
print('с оптимизациями:')

print(expression.diff('x').optim())
print(expression.diff('y').optim())

print('дважды производная:')
print(expression.diff('x').diff('x').optim())


print('упаковка в "программы"')

prog = program({'output':expression})
prog(x = 1, y = 2)


print('порядок выполнения:')
print(prog)

((x^2)+y*x+cos(x)+3)
без оптимизаций:
(2*(x^1)*1.0+(0.0*x+y*1.0)+(-sin(x))*1.0+0.0)
(2*(x^1)*0.0+(1.0*x+y*0.0)+(-sin(x))*0.0+0.0)
с оптимизациями:
(x*2+y+sin(x)*-1)
x
дважды производная:
((1^2)*(x^0)*2+cos(x)*-1)
упаковка в "программы"
порядок выполнения:
x,y,3
(x^2),y*x,cos(x)
((x^2)+y*x+cos(x)+3)



## Basic vector operations

In [3]:
from diffeq.utils.vectors import *
A = vector(x = 10, y = 10)
B = vector(x = 3, y = -1, z = 1)

print('A:', A, sep = '\n'*2)
print('B:', B, sep = '\n'*2)
print('A + B:', A + B, sep = '\n'*2)
print('A - B:', A - B, sep = '\n'*2)
print('A @ B:', A @ B, sep = '\n'*2)
print('A * B:', A * B, sep = '\n'*2)
print('A / B:', A / B, sep = '\n'*2)
print('A ** B:', A ** B, sep = '\n'*2)
print('A**2:', A**2, sep = '\n'*2)
print('2**A:', 2**A, sep = '\n'*2)
print('A@A:', A@A)

A:

┌─────────────┐
│axis │value  │
├─────┼───────┤
│x    │10     │
│y    │10     │
└─────────────┘
B:

┌─────────────┐
│axis │value  │
├─────┼───────┤
│x    │3      │
│y    │-1     │
│z    │1      │
└─────────────┘
A + B:

┌──────────────┐
│axis │value   │
├─────┼────────┤
│y    │9       │
│z    │1.0     │
│x    │13      │
└──────────────┘
A - B:

┌───────────────┐
│axis │value    │
├─────┼─────────┤
│y    │11       │
│z    │-1.0     │
│x    │7        │
└───────────────┘
A @ B:

20
A * B:

┌──────────────┐
│axis │value   │
├─────┼────────┤
│y    │-10     │
│z    │0.0     │
│x    │30      │
└──────────────┘
A / B:

┌─────────────────────────────┐
│axis │value                  │
├─────┼───────────────────────┤
│y    │-10.0                  │
│z    │0.0                    │
│x    │3.3333333333333335     │
└─────────────────────────────┘
A ** B:

┌───────────────┐
│axis │value    │
├─────┼─────────┤
│y    │0.1      │
│z    │0.0      │
│x    │1000     │
└───────────────┘
A**2:

┌──────────

## Vector functions

In [4]:
@vector_function
def foo(x, y):
    return vector(
        x=10*y*x + y,
        y=x,
    )

v = vector(x = 10, y = 11)
print('value of input vector: ', v, sep = '\n')
print('function form: ', sep = '\n')
print(foo)
print('function output: ', sep = '\n')
print(foo(v))
print("function's yacobian: ", sep = '\n')
print(foo.yacobian)
print("function's yacobian value: ", sep = '\n')
print(foo.yacobian(v))

value of input vector: 
┌─────────────┐
│axis │value  │
├─────┼───────┤
│x    │10     │
│y    │11     │
└─────────────┘
function form: 
┌────────────────────────┐
│axis │function          │
├─────┼──────────────────┤
│x    │(y*x*10+y)        │
│y    │x                 │
└────────────────────────┘
function output: 
┌───────────────┐
│axis │value    │
├─────┼─────────┤
│x    │1111     │
│y    │10       │
└───────────────┘
function's yacobian: 
┌────────────────────────────┐
│axis     │function          │
├─────────┼──────────────────┤
│dx_dy    │(x*10+1.0)        │
│dx_dx    │(y*10)            │
│dy_dy    │0.0               │
│dy_dx    │1.0               │
└────────────────────────────┘
function's yacobian value: 
┌────────────────────┐
│axis     │value     │
├─────────┼──────────┤
│dx_dy    │101.0     │
│dx_dx    │110       │
│dy_dy    │0.0       │
│dy_dx    │1.0       │
└────────────────────┘


composition:

In [5]:
@vector_function
def foo1(x, y):
    return vector(
        x=10*y*x + y,
        y=-x + (-1),
    )

@vector_function
def foo2(x, y):
    return vector(
        x = 3*y - x,
        y = x - 1j
    )

In [6]:
@vector_function
def composed(x, y):
    return foo2(foo1(vector(x = x, y = y)))

A = vector(x = 1, y = 2)
print(composed)
print(composed(A))
print(foo2(foo1(A)))

print(composed.yacobian)

┌─────────────────────────────────────────┐
│axis │function                           │
├─────┼───────────────────────────────────┤
│x    │(((-x)+-1)*3+(-(y*x*10+y)))        │
│y    │(y*x*10+y+-1j)                     │
└─────────────────────────────────────────┘
┌──────────────────┐
│axis │value       │
├─────┼────────────┤
│x    │-28         │
│y    │(22-1j)     │
└──────────────────┘
┌──────────────────┐
│axis │value       │
├─────┼────────────┤
│x    │-28         │
│y    │(22-1j)     │
└──────────────────┘
┌───────────────────────────────────┐
│axis     │function                 │
├─────────┼─────────────────────────┤
│dx_dy    │((-(x*10+1.0))+3)        │
│dx_dx    │((-y*10)+-3.0)           │
│dy_dy    │(x*10+1.0)               │
│dy_dx    │(y*10)                   │
└───────────────────────────────────┘


In [7]:
print(composed.div)

┌───────────────────────────────────┐
│axis   │function                   │
├───────┼───────────────────────────┤
│div    │(x*10+(-y*10)+-2.0)        │
└───────────────────────────────────┘


In [8]:
print(composed.div.yacobian)

┌───────────────────────┐
│axis       │function   │
├───────────┼───────────┤
│ddiv_dy    │-10        │
│ddiv_dx    │10         │
└───────────────────────┘


## System of differential equations

In [9]:
solver = rk4_integrator(0.1)
sys = system(
    vector_function(lambda x, y:vector(x = -y, y = x)), solver, initials = vector(x = 1, y = 0)
)
results = sys.run(10)

In [10]:
print(results)

┌────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┐
│axis    │value                                                                                                                                                                                              │
├────────┼───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┤
│x       │[1, 0.540302967116884, -0.41614526873411317, -0.9899919390660014, -0.6536459531899061, 0.28365810583410267, 0.9601684949770736, 0.7539057070501398, -0.14549338089595665, -0.9111266132598114]     │
│y       │[0, 0.8414704778002743, 0.9092979917935007, 0.1411224448434349, -0.7568001143085917, -0.9589251198182553, -0.27942016563257394, 0.6569818976735576, 0.989358664182

# Jacobian and Divergence 

In [11]:
# Define a vector field: F(x,y) = [x*y, x+y]
def field(state):
    x, y = state['x'], state['y']
    return vector({'x': x*y, 'y': x + y})

# Create vector function object
F = vector_function(lambda x, y: vector({'x': x*y, 'y': x + y}))

# Point to evaluate at
point = vector({'x': 2.0, 'y': 3.0})

# Compute Jacobian numerically
J_num = jacobian(field, point)
print("Numerical Jacobian:")
print(J_num)

# Compute divergence numerically
div_num = divergence(field, point)
print(f"\nNumerical Divergence: {div_num}")

Numerical Jacobian:
┌─────────────────────────────────┐
│axis     │value                  │
├─────────┼───────────────────────┤
│dx_dx    │3.0000000001232743     │
│dx_dy    │1.9999999999834963     │
│dy_dx    │0.9999999998437185     │
│dy_dy    │0.9999999998437185     │
└─────────────────────────────────┘

Numerical Divergence: 3.9999999999669926


## Visualization

In [12]:
import random as randm
from random import gauss, random
import diffeq.plotting.interactive as interactive
from diffeq import *
solver = rk4_integrator(0.01, 1)

In [13]:
randm.seed(42)

### Lorenz's system

In [14]:
# Lorenz Attractor
a, b, c = -15, 35, -3/2
lorenz_sys = system(
    vector_function(lambda x, y, z:vector(x = a*(x - y), y = b*x - y - z*x, z = x*y + c*z)), solver
    )
lorenz_trjs = []
for _ in range(10):
    lorenz_sys.state = vector(x = gauss(), y = gauss(), z = gauss())
    results = lorenz_sys.run(10)
    lorenz_trjs.append(results)
out = interactive.generate_html(lorenz_trjs, ('x', 'y', 'z'), color = interactive.start_end_grad(), title = 'Lorenz Attractor', path = 'output/lorenz.html')

### Rössler Attractor

In [15]:
# Rössler Attractor
a, b, c = 0.2, 0.2, 7.7
rossler_sys = system(
    vector_function(lambda x, y, z:vector(x = -y - z, y = x + a*y, z = b + z*(x - c))), solver
    )
rossler_trjs = []
for _ in range(10):
    rossler_sys.state = vector(x = gauss(), y = gauss(), z = gauss())*3
    results = rossler_sys.run(10)
    rossler_trjs.append(results)
out = interactive.generate_html(rossler_trjs, ('x', 'y', 'z'), color = interactive.start_end_grad(), title = 'Rossler Attractor', path = 'output/rossler.html')

### Lu Chen Attractor

In [16]:
# Lu Chen Attractor
a, b, c, u = 36, 3, 20, -3
multiscroll_sys = system(
    vector_function(lambda x, y, z:vector(x = a*(y - x), y = x - x*z + c*y + u, z = x*y-b*z)), solver
    )
multiscroll_trjs = []
for _ in range(10):
    multiscroll_sys.state = vector(x = gauss(), y = gauss(), z = gauss()) + vector(x = 0.1, y = 0.3, z = -0.6)
    results = multiscroll_sys.run(10)
    multiscroll_trjs.append(results)
out = interactive.generate_html(multiscroll_trjs, ('x', 'y', 'z'), color = interactive.start_end_grad(), title = 'Lu Chen Attractor', path = 'output/Lu Chen.html')

### Trillium Attractor

In [17]:
# Trillium Attractor
a, b, c, d = 0.1, 0.1, 14, 0.08
trillium_sys = system(
    vector_function(lambda x, y, z:10*vector(x = a*x - b*y*z, y = -c*y + x*z, z = -d*z + x*y)), solver
    )
trillium_trjs = []
for _ in range(50):
    trillium_sys.state = vector(x = gauss(), y = gauss(), z = gauss())*2
    results = trillium_sys.run(10)
    trillium_trjs.append(results)
out = interactive.generate_html(trillium_trjs, ('x', 'y', 'z'), color = interactive.start_end_grad(), title = 'Trillium Attractor', path = 'output/trillium.html')

### Tomas Attractor

In [18]:
# Tomas Attractor
a, b = 0.2, 1.0
tomas_sys = system(
    vector_function(lambda x, y, z:vector(x = -a*x + b*symb.sin(y), y = -a*y + b*symb.sin(z), z = -a*z + b*symb.sin(x))), solver
    )
tomas_trjs = []
for _ in range(10): 
    tomas_sys.state = 5*(vector(x = random(), y = random(), z = random())*2 - 1.0)
    results = tomas_sys.run(100)
    tomas_trjs.append(results)
out = interactive.generate_html(tomas_trjs, ('x', 'y', 'z'), color = interactive.start_end_grad(), title = 'Tomas Attractor', path = 'output/tomas.html')

### linear system

In [19]:
# just linear system
linear_sys = system(
    vector_function(lambda x, y, z:vector(x = -y, y = x, z = -0.3*z)), solver
    )
linear_trjs = []
for _ in range(50):
    linear_sys.state = vector(x = gauss(), y = gauss(), z = gauss())*2
    results = linear_sys.run(10)
    linear_trjs.append(results)
out = interactive.generate_html(linear_trjs, ('x', 'y', 'z'), color = interactive.start_end_grad(), title = 'linear system', path = 'output/linear.html')